In [81]:

import gensim
from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)
import nltk
from textProcessing import *

In [119]:
path = 'C:\\Users\\hp\\Desktop\\college-5-2\\IR\\lotte\\lifestyle\\dev\\collection.tsv'

text = textProcessing.readDataset(path)
text.columns=['text']
text=text['text']
text = textProcessing.normalization(text.head(500))
text = textProcessing.toLowercase(text)
text = textProcessing.delete_url(text)
text = textProcessing.delete_punctuation(text)
text = textProcessing.delete_dates_and_times(text)
text =  textProcessing.delete_numbers(text)
text = text.apply(textProcessing.tokenize_and_remove_stopwords)
texts = text.apply(textProcessing.lemmatization)

In [56]:

# create a dictionary from the text data
dictionary = corpora.Dictionary(texts)

# create a corpus from the text data
corpus = [dictionary.doc2bow(text) for text in texts]

# train the LDA model on the corpus
ldamodel = models.LdaModel(corpus, num_topics=5, id2word=dictionary)

# extract the topics from the model
topics = ldamodel.print_topics(num_topics=5, num_words=5)

# print the topics
for topic in topics:
    print(topic)

# print(corpus)

(0, '0.039*"dog" + 0.009*"dollar" + 0.009*"pet" + 0.006*"food" + 0.006*"one"')
(1, '0.016*"dog" + 0.012*"cat" + 0.010*"dollar" + 0.009*"pet" + 0.008*"baby"')
(2, '0.025*"dog" + 0.013*"cat" + 0.010*"dollar" + 0.006*"one" + 0.005*"pet"')
(3, '0.016*"cat" + 0.014*"dog" + 0.011*"rabbit" + 0.009*"litter" + 0.007*"time"')
(4, '0.019*"dog" + 0.013*"cat" + 0.009*"dollar" + 0.007*"like" + 0.007*"time"')


In [61]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in texts]

lda_model = gensim.models.LdaMulticore(bow_corpus, 
 num_topics = 8, 
 id2word = dictionary, 
 passes = 10,
 workers = 2)

for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} nWords: {}".format(idx, topic ))
    print("n")

Topic: 0 nWords: 0.097*"cat" + 0.022*"may" + 0.020*"like" + 0.016*"help" + 0.015*"also" + 0.015*"old" + 0.013*"use" + 0.013*"something" + 0.013*"small" + 0.012*"toy"
n
Topic: 1 nWords: 0.047*"pet" + 0.023*"use" + 0.023*"water" + 0.019*"treat" + 0.018*"keep" + 0.018*"age" + 0.017*"like" + 0.017*"need" + 0.016*"behavior" + 0.016*"used"
n
Topic: 2 nWords: 0.031*"food" + 0.024*"time" + 0.018*"case" + 0.016*"keep" + 0.016*"away" + 0.015*"would" + 0.014*"need" + 0.014*"likely" + 0.013*"problem" + 0.012*"try"
n
Topic: 3 nWords: 0.043*"cat" + 0.040*"food" + 0.031*"fence" + 0.021*"would" + 0.021*"one" + 0.019*"even" + 0.015*"eat" + 0.014*"water" + 0.014*"way" + 0.012*"ive"
n
Topic: 4 nWords: 0.035*"pet" + 0.024*"child" + 0.019*"get" + 0.018*"one" + 0.018*"time" + 0.016*"like" + 0.016*"puppy" + 0.014*"take" + 0.014*"also" + 0.013*"dont"
n
Topic: 5 nWords: 0.052*"fish" + 0.028*"small" + 0.026*"volume" + 0.020*"start" + 0.020*"puppy" + 0.020*"noise" + 0.019*"need" + 0.019*"lot" + 0.019*"reward" + 

In [123]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora, models, similarities


# Create dictionary and corpus
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(doc) for doc in texts]

# Train the LDA model
lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=10)

# Create a similarity index for the trained model
index = similarities.MatrixSimilarity(lda_model[corpus])

# Convert the query to the LDA vector representation
query_vector = dictionary.doc2bow(['give cat pill'])
query_lda_vector = lda_model[query_vector]

# Calculate the similarity between the query and each topic
lda_index = similarities.MatrixSimilarity(lda_model[corpus])
sims = lda_index[query_lda_vector]


# Sort the topics by similarity score (descending order)
sorted_topics = sorted(enumerate(sims), key=lambda item: -item[1])

# Print the most relevant topic
top_topic_index, top_topic_score = sorted_topics[0]
top_topic = lda_model.print_topic(top_topic_index)


print("Most Relevant Topic:")
print(top_topic)

IndexError: index 374 is out of bounds for axis 0 with size 3

In [85]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora, models
from sklearn.metrics import jaccard_score



# Create dictionary and corpus
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(doc) for doc in texts]

# Train the LDA model
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

# Topic ID to retrieve document IDs for
topic_id = 0  # Replace with the desired topic ID

# Retrieve document IDs for the given topic
topic_docs = [doc_id for doc_id, doc_topic in enumerate(lda_model[corpus]) if max(doc_topic, key=lambda item: item[1])[0] == topic_id]

# Print the document IDs for the topic
print("Document IDs for Topic", topic_id, ":")
print(topic_docs)


# Preprocess the query
query_tokens = ['give cat pill']

# Convert the query to the binary vector representation
query_vector = [1 if word in query_tokens else 0 for word in dictionary.token2id]

# Calculate the Jaccard similarity between the query and each topic
similarity_scores = []
for topic_id in range(lda_model.num_topics):
    topic_vector = lda_model.get_topic_terms(topic_id)
    topic_terms = set([dictionary[id] for id, _ in topic_vector])
    topic_vector = [1 if word in topic_terms else 0 for word in dictionary.token2id]
    similarity_score = jaccard_score(query_vector, topic_vector)
    similarity_scores.append(similarity_score)

# Sort the topics by similarity score (descending order)
sorted_topics = sorted(enumerate(similarity_scores), key=lambda item: -item[1])

# Print the similarity scores
print("Similarity Scores:")
for topic_id, similarity_score in sorted_topics:
    print("Topic", topic_id, ":", similarity_score)

Document IDs for Topic 0 :
[2, 9, 28, 36, 38, 44, 46, 54, 67, 68, 74, 84, 86, 87, 89, 95, 101, 118, 143, 145, 148, 153, 183, 186, 189, 196, 198, 227, 230, 233, 234, 236, 261, 264, 281, 283, 285, 287, 288, 292, 302, 303, 305, 310, 327, 332, 368, 373, 375, 389, 400, 409, 415, 451, 454, 465, 470, 478]
Similarity Scores:
Topic 0 : 0.0
Topic 1 : 0.0
Topic 2 : 0.0
Topic 3 : 0.0
Topic 4 : 0.0


In [106]:
texts.to_csv('C:\\Users\\hp\\Desktop\\lemmetized.csv', sep='\t', index=False, encoding='utf-8')

In [109]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# texts = pd.read_csv('C:\\Users\\hp\\Desktop\\lemmetized.csv' ,encoding='utf-8' ,sep='\t')

path = 'C:\\Users\\hp\\Desktop\\college-5-2\\IR\\lotte\\lifestyle\\dev\\collection.tsv'

corpus = textProcessing.readDataset(path)
corpus.columns =  ['text']
corpus = corpus['text']

vectorizer = pickle.load(open("C:\\Users\\hp\\Desktop\\college-5-2\\IR\\ir_project\\tfidf_output\\model.pickle", "rb"))
X = pickle.load(open("C:\\Users\\hp\\Desktop\\college-5-2\\IR\\ir_project\\tfidf_output\\tfidf_docs.pickle", "rb"))

# Ensure the corpus and TF-IDF matrix are aligned
assert len(corpus) == X.shape[0], "Corpus and TF-IDF matrix size mismatch"

# Fit the LDA model
lda = LatentDirichletAllocation(n_components=12)
lda.fit(X)

# Transform documents to topic distribution
doc_topic_dist = lda.transform(X)

# Create an index
index = {}
for i, doc in enumerate(corpus):
    index[doc] = doc_topic_dist[i]

# Example query

# # Calculate similarity between query and documents
# similarity = cosine_similarity(query_topic_dist, doc_topic_dist)

# # Rank documents
# ranked_docs = similarity.argsort()[0][::-1]

# # Display results
# for idx in ranked_docs:
#     print(idx)


In [124]:
print(index)

MatrixSimilarity<500 docs, 3 features>


In [115]:
query_vec = vectorizer.transform(['give cat pill'])
query_topic_dist = lda.transform(query_vec)

In [116]:
model_output= []
cos=cosine_similarity(query_topic_dist,doc_topic_dist).flatten()
map=dict()
for x in range(cos.size):
    map[cos[x]]=x

myKey = list(map.keys())
myKey.sort(reverse=True)
for i in range(10):
    model_output.append(map.get(myKey[i]))
    
print(model_output)

[152839, 263354, 226768, 241838, 44022, 42149, 259511, 252432, 150214, 123637]


In [104]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora, models
from sklearn.metrics import jaccard_score
import numpy as np


# Create dictionary and corpus
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(doc) for doc in texts]

# Train the LDA model
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

# Topic ID to retrieve document IDs for
topic_id = 0  # Replace with the desired topic ID

# Retrieve document IDs for the given topic
topic_docs = [doc_id for doc_id, doc_topic in enumerate(lda_model[corpus]) if max(doc_topic, key=lambda item: item[1])[0] == topic_id]

# Print the document IDs for the topic
print("Document IDs for Topic", topic_id, ":")
print(topic_docs)

# Preprocess the query
query_tokens = ["give away light cyclist light"]

# Convert the query to the binary vector representation
query_vector = [1 if word in query_tokens else 0 for word in dictionary.token2id]

# Calculate the Jaccard similarity between the query and each topic
similarity_scores = []
for topic_id in range(lda_model.num_topics):
    topic_vector = lda_model.get_topic_terms(topic_id, topn=len(dictionary))
    topic_terms = set([dictionary[id] for id, _ in topic_vector])
    topic_vector = [1 if word in topic_terms else 0 for word in dictionary.token2id]
    similarity_score = jaccard_score(query_vector, topic_vector)
    similarity_scores.append(similarity_score)

# Sort the topics by similarity score (descending order)
sorted_topics = sorted(enumerate(similarity_scores), key=lambda item: -item[1])

# Print the similarity scores
print("Similarity Scores:")
for topic_id, similarity_score in sorted_topics:
    print("Topic", topic_id, ":", similarity_score)

# Output the most similar topic for the query
most_similar_topic = sorted_topics[0][0]
print(f"The query is most similar to Topic {most_similar_topic} with a similarity score of {sorted_topics[0][1]}")

Document IDs for Topic 0 :
[7, 11, 13, 15, 16, 17, 22, 25, 27, 29, 34, 40, 42, 44, 45, 48, 51, 55, 66, 74, 81, 82, 92, 97, 99, 100, 104, 109, 115, 116, 117, 124, 127, 129, 130, 137, 148, 151, 156, 157, 159, 161, 163, 169, 170, 171, 186, 191, 193, 195, 200, 223, 225, 231, 243, 247, 253, 254, 264, 265, 268, 269, 274, 293, 300, 313, 314, 317, 318, 320, 327, 337, 338, 340, 343, 348, 349, 350, 352, 353, 354, 355, 356, 357, 360, 371, 373, 376, 377, 379, 380, 385, 386, 398, 399, 404, 405, 408, 411, 413, 415, 425, 427, 428, 430, 439, 454, 468, 473, 479, 482, 486, 489, 491, 493, 495, 496, 498, 499]
Similarity Scores:
Topic 0 : 0.0
Topic 1 : 0.0
Topic 2 : 0.0
Topic 3 : 0.0
Topic 4 : 0.0
The query is most similar to Topic 0 with a similarity score of 0.0
